In [ ]:
__author__ = 'Alice Jacques <alice.jacques@noirlab.edu>, Stephanie Juneau <stephanie.juneau@noirlab.edu>, SPARCL team'
__version__ = '20241213' # yyyymmdd
__datasets__ = ['sdss_dr16', 'boss_dr16', 'desi_edr', 'desi_dr1']
__keywords__ = ['sparcl', 'spectroscopy', 'HowTo', 'sdss spectra', 'desi spectra', 'tutorial']

# How to use SPARCL at NERSC
## SPectra Analysis and Retrievable Catalog Lab (SPARCL)
Alice Jacques (NOIRLab), Stéphanie Juneau (NOIRLab), Benjamin Weaver (NOIRLab), Steve Pothier (NOIRLab), Adam Bolton (SLAC) and the SPARCL team

### Table of contents
* [Goals & Summary](#goalssummary)
* [Disclaimer & attribution](#disclaimer)
* [If necessary, install the most recent version of the SPARCL Client](#install)
* [Imports and setup](#imports)
* [Authentication for SPARCL](#auth_sparcl)
* [Data sets available in SPARCL](#datasets)
* [Get default field names](#defaultfieldnames)
* [Get all field names](#allfieldnames)
* [Data discovery I: using SPARCL's <tt>client.find()</tt> method](#datadiscoveryI)
* [Data discovery II: using the DESI DR1 database](#datadiscoveryII)
* [Retrieve records by <tt>specid</tt> using <tt>client.retrieve_by_specid()</tt>](#retrieve_specid)
* [Reorder spectrum records](#reorder)
* [Convert retrieved output to a Spectrum1D object](#convert)
* [Plot spectra](#plot)
* [Additional acknowledgments](#additionalacknowledgments)

<a class="anchor" id="goalssummary"></a>
## Goals & Summary 

SPARCL (SPectra Analysis and Retrievable Catalog Lab) is an online service for discovery and retrieval of one-dimensional optical-infrared spectra. SPARCL is designed to work for large survey datasets containing many millions of spectra, and to provide access to multiple different data sets through common methods. For more information, see the [SPARCL User Guide](https://astrosparcl.datalab.noirlab.edu/static/SPARCLUserManual.pdf).

This notebook provides a basic introduction to using the SPARCL client (or sparclclient) to find and retrieve spectroscopic data within a Python notebook context. The sparclclient connects to the SPARCL server at [NSF NOIRLab](https://noirlab.edu/public/) and provides access to the contents of the SPARCL database.

To see the current data sets and number of spectra available in the SPARCL database, please visit the [SPARCL summary page](https://astrosparcl.datalab.noirlab.edu/sparc/).

To get the latest sparclclient documentation, visit the [sparclclient readthedocs site](https://sparclclient.readthedocs.io/en/latest/).

Feedback on SPARCL (questions, comments, science use cases, feature requests, bug reports, confusing error messages, etc.) can be submitted to datalab-spectro@noirlab.edu. For bug reports and confusing error messages, it's helpful if you include: a log of *what you did*, *the result you got*, and *the result you expected*.

See our science use-case notebooks that use SPARCL:
- [Introduction to DESI Early Data Release (EDR) at the Astro Data Lab](https://github.com/astro-datalab/notebooks-latest/blob/master/03_ScienceExamples/DESI/01_Intro_to_DESI_EDR.ipynb)
- [Comparing SDSS and DESI spectra using SPARCL](https://github.com/astro-datalab/notebooks-latest/blob/master/03_ScienceExamples/DESI/02_DESI_EDR_SDSS_Comparison.ipynb)
- [Stacking SDSS Spectra of Galaxies Selected from the BPT Diagram](https://github.com/astro-datalab/notebooks-latest/blob/master/03_ScienceExamples/EmLineGalaxies/01_EmLineGalaxies_SpectraStack.ipynb)
- [Multi-wavelength Image Cutouts and SDSS Spectra of Active Galaxies with Extreme Emission-Line Ratios](https://github.com/astro-datalab/notebooks-latest/blob/master/03_ScienceExamples/EmLineGalaxies/02_EmLineGalaxies_Outliers.ipynb)

And our other How-To notebooks that use SPARCL:
- [Obtain spectra with SPARCL and plot them with Jdaviz](https://github.com/astro-datalab/notebooks-latest/blob/master/04_HowTos/SPARCL/Plot_SPARCL_Spectra_with_Jdaviz.ipynb)
- [Obtain spectra with SPARCL and plot them with prospect](https://github.com/astro-datalab/notebooks-latest/blob/master/04_HowTos/SPARCL/Plot_SPARCL_Spectra_with_Prospect.ipynb)

<a class="anchor" id="attribution"></a>
# Disclaimer & attribution

Disclaimers
-----------
Note that using the Astro Data Lab and/or SPARCL constitutes your agreement with our minimal [Disclaimers](https://datalab.noirlab.edu/disclaimers.php).

Acknowledgments
---------------
Researchers are required to acknowlege the source of the data (DESI and/or SDSS) in their papers following each survey's guidelines. If you use **SPARCL** in your published research, also include this text in your paper's Acknowledgments section:

_This research uses services or data provided by the SPectra Analysis and Retrievable Catalog Lab (SPARCL), which is part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

If you publish in an AAS journal, please add `\software{SPARCL}` and cite:
* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576 

If you are using the Astro Data Lab in your research, also see the [Additional acknowledgments](#additionalacknowledgments) section. The NOIRLab Library maintains [lists of proper acknowledgments](https://noirlab.edu/science/about/scientific-acknowledgments) to use when publishing papers using the Lab's facilities, data, or services.

<a class="anchor" id="install"></a>
### If necessary, install the most recent version of the SPARCL Client:
If you are using the Astro Data Lab Jupyter notebook server, you do not need to run this cell.

<div class="alert alert-block alert-info">
<b>Reminder:</b> After installing the most recent version, please restart your kernel
</div>

In [ ]:
## Uncomment the following only if SPARCL client is not already installed
#!pip install --user --upgrade sparclclient

<a class="anchor" id="imports"></a>
## Imports and setup

In [ ]:
# SPARCL imports
from sparcl.client import SparclClient

# 3rd party imports
import numpy as np
import astropy.units as u
from astropy.table import Table
from specutils import Spectrum1D
from astropy.nddata import InverseVariance
from astropy.convolution import convolve, Gaussian1DKernel
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from getpass import getpass

# For NERSC specprod database access (other notebook)
from desiutil.log import get_logger, DEBUG
import specprodDB.load as db

# plots default setup
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = (14,8)

In [ ]:
# NERSC database logger
db.log = get_logger()

#### We now create a SPARCL client instance:

In [ ]:
client = SparclClient()
client

<a class="anchor" id="auth_sparcl"></a>
# Authentication for SPARCL
All public SPARCL data sets can be accessed without explicitly logging in. However, some data sets are private and can only be accessed by authorized users. If you are an authorized user and wish to login to SPARCL, un-comment the cell below and enter your NOIRLab CSDC SSO user name and password. If you need to create an account, sign-up at https://sso.csdc.noirlab.edu/account/signup/. If you encounter an issue, email datalab-spectro@noirlab.edu with your First Name, Last Name, and Email Address (the same one you used to create your NOIRLab CSDC SSO account).

To logout of SPARCL after a session, use:
```
client.logout()
```

For assistance with SPARCL authentication/authorization please contact us at datalab-spectro@noirlab.edu

In [ ]:
client.login(input("Enter SSO user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))

#### View which data sets you have access to:
**Note:** if you are not logged in or if your SSO user name is not in the authorized list of SPARCL users, you will only see public data sets

In [ ]:
client.authorized

<a class="anchor" id="datasets"></a>
## Data sets available in SPARCL
List all currently available data sets from the server/url associated with client

In [ ]:
client.all_datasets

<a class="anchor" id="defaultfieldnames"></a>
## Get default field names
Gets fields tagged as 'default' that are common to all data sets in the `dataset_list` passed to the function. If `dataset_list` is None (the default), the function returns the intersection of 'default' fields across all datasets currently available in the SPARC database.

In [ ]:
client.get_default_fields?

In [ ]:
client.get_default_fields()

<a class="anchor" id="allfieldnames"></a>
## Get all field names
Gets fields tagged as 'all' that are common to all data sets in the `dataset_list` passed to the function. If `dataset_list` is None (the default), the function returns the intersection of 'all' fields across all datasets currently available in the SPARC database.

In [ ]:
client.get_all_fields?

In [ ]:
print(sorted(client.get_all_fields()))

In [ ]:
print(sorted(client.get_all_fields(dataset_list=['SDSS-DR16', 'BOSS-DR16'])))

In [ ]:
fields_EDR = sorted(client.get_all_fields(dataset_list=['DESI-EDR']))

print(fields_EDR)

In [ ]:
fields_DR1 = sorted(client.get_all_fields(dataset_list=['DESI-DR1']))

print(fields_DR1)

In [ ]:
## Fields in EDR that are not in DR1
set(fields_EDR).difference(fields_DR1)

In [ ]:
## Fields in DR1 that are not in EDR
set(fields_DR1).difference(fields_EDR)

<a class="anchor" id="datadiscoveryI"></a>
## Data discovery I: using SPARCL's `client.find()` method
The first way you can discover your data is by using SPARCL's `client.find()` method, which allows you to find records in the SPARCL database based on certain parameters passed to the function. Only Core fields may be in the `outfields` and `constraints` parameters. The descriptions for all fields, including Core fields, is located [here](https://astrosparcl.datalab.noirlab.edu/sparc/sfc/). The SPARCL Core fields constraint types are:


| Field name       | Constraint type | Example |
|:----------------|:---------------|:-------|
| data_release     | List of allowed values<br>from [SPARCL Categoricals](https://astrosparcl.datalab.noirlab.edu/sparc/cats/) | ['DESI-EDR', 'BOSS-DR16', 'SDSS-DR16']
| datasetgroup     | List of allowed values<br>from [SPARCL Categoricals](https://astrosparcl.datalab.noirlab.edu/sparc/cats/) | ['DESI', 'SDSS_BOSS']
| dateobs_center   | Range of values | ['2013-03-14T10:16:17Z',<br>'2014-05-24T12:10:00Z']
| dec              | Range of values | [2.03, 7.76]
| exptime          | Range of values | [3603.46, 3810.12]
| instrument       | List of allowed values<br>from [SPARCL Categoricals](https://astrosparcl.datalab.noirlab.edu/sparc/cats/) | ['SDSS', 'BOSS', 'DESI']
| ra               | Range of values (may not<br>"wrap" around RA=0) | [44.53, 47.96]
| redshift         | Range of values | [0.5, 0.9]
| redshift_err     | Range of values | [0.000225, 0.000516]
| redshift_warning | List of values  | [0, 3, 5]
| sparcl_id               | List of values (but not<br>intended for data discovery) | ['00001658-460c-4da1-987d-e493d8c9b89b',<br>'000017b6-56a2-4f87-8828-3a3409ba1083']
| site             | List of allowed values<br>from [SPARCL Categoricals](https://astrosparcl.datalab.noirlab.edu/sparc/cats/) |  ['apo', 'kpno']
| specid           | List of values | [6988698046080241664, 6971782884823945216]
| spectype         | List of allowed values<br>from [SPARCL Categoricals](https://astrosparcl.datalab.noirlab.edu/sparc/cats/) | ['GALAXY', 'STAR', 'QSO']
| specprimary      | List of values (but typically<br>would only include 1 if<br>being used for data<br>discovery constraints) | [1]
| targetid         | List of values | [1237679502171374316, 1237678619584692841]
| telescope        | List of allowed values<br>from [SPARCL Categoricals](https://astrosparcl.datalab.noirlab.edu/sparc/cats/) | ['sloan25m', 'kp4m']
| wavemin          | Range of values | [3607, 3608]
| wavemax          | Range of values | [10363, 10364]

In [ ]:
client.find?

#### Define the fields we want returned (`outfields`) and the constraints (`constraints`):

In [ ]:
out = ['sparcl_id', 'ra', 'dec', 'redshift', 'spectype', 'data_release', 'redshift_err']
cons = {'spectype': ['GALAXY'],
        'redshift': [0.5, 0.9]}

#### Execute the `client.find()` method with our parameters:
The `limit` argument here is being used for demonstration purposes only, and simply returns only the first 20 results here.

<div class="alert alert-block alert-warning">
<b>Important:</b> the `limit` parameter will default to 500 if not otherwise specified. 
</div>

In [ ]:
found_I = client.find(outfields=out, constraints=cons, limit=20)
["%s, %s, %f, %f, %f" % (f.sparcl_id, f.data_release, f.ra, f.redshift, f.redshift_err)
 for f in found_I.records]

<a class="anchor" id="datadiscoveryII"></a>
## Data discovery II: using DESI DR1 (iron) database at NERSC
The second way you can discover your data is by using DESI DR1 catalogs and databases hosted at NERSC for DESI DR1 (iron). We will focus on the `zpix` catalog for the healpix-based coadded spectra that are in SPARCL. For large queries, this method will work *faster* than using `client.find()` when querying the database. We will work with this method for the remainder of this notebook.

The connection to the DR1 database requires a `~/.pgpass` file to store connection credentials. The following code will attempt to connect to the database.  If it fails, it prints the
commands to update your `~/.pgpass` file (but it won't do that automatically in case you did a Jupyter "run all cells").

In [ ]:
try:
    postgresql = db.setup_db(schema='iron', hostname='specprod-db.desi.lbl.gov', username='desi_public')
except RuntimeError:
    print("\nIt looks like you need to add access credentials to your $HOME/.pgpass file.")
    print("Try running this from the command line and then rerun this cell.")
    print()
    print("cat /global/common/software/desi/desi_public.pgpass >> ~/.pgpass; chmod 600 ~/.pgpass")

As an example, let's look for a somewhat rare case of low-redshift (`0.3<z<0.4`) quasars (`spectype='QSO'`) and only keep 20 results (`[:20]`).

In [ ]:
%%time
q = db.dbSession.query(db.Zpix.targetid, db.Zpix.survey, db.Zpix.program,
                       db.Zpix.desi_target,
                       db.Zpix.z, db.Zpix.zwarn, db.Zpix.spectype,
                       db.Zpix.main_primary, db.Zpix.zcat_primary).filter(db.Zpix.zwarn == 0)\
                                                                  .filter(db.Zpix.main_primary)\
                                                                  .filter(db.Zpix.zcat_primary)\
                                                                  .filter(db.Zpix.z > 0.3)\
                                                                  .filter(db.Zpix.z < 0.4)\
                                                                  .filter(db.Zpix.spectype == 'QSO')\
                                                                  .filter(db.Zpix.program == 'dark')[:20]

## Query constraints
# (db.Zpix.zwarn == 0): ZWARN=0 means no warning on the Redrock redshift
# (db.Zpix.main_primary): MAIN_PRIMARY=True for the best spectrum in the Main survey
# (db.Zpix.zcat_primary): ZCAT_PRIMARY=True for the best spectrum in the Zpix catalog
# (db.Zpix.z > 0.3): Redshift cut Z>0.3
# (db.Zpix.z < 0.4): Redshift cut Z<0.4
# (db.Zpix.spectype == 'QSO'): Redrock spectral type SPECTYPE='QSO'
# (db.Zpix.program == 'dark'): PROGRAM='dark' (dark-time spectra tend to have higher S/N)

In [ ]:
## For convenience later to join with the output spectra, we convert the query results (q)
## into a Pandas DataFrame (df)
found_df_II = pd.DataFrame(data=q)

In [ ]:
print('Number of returned results from the database query = ', len(found_df_II))
found_df_II

<a class="anchor" id="retrieve_specid"></a>
## Retrieve records by `specid` using `client.retrieve_by_specid()`
In order to retrieve spectra records from SPARCL by `specid`, pass the following to the `client.retrieve_by_specid()` method:

`specid_list` : List of specIDs.  
`dataset_list` : List of data sets to search for the specIDs in (default: None, which will search all available data sets).  
`include` : List of field names to include in each record (default: 'DEFAULT').  
`limit` : Maximum number of records to return (default: 500). Max allowed is 24,000.

<div class="alert alert-block alert-warning">
<b>Important:</b> A reasonable number of DESI records to request retrieval of is up to about 10,000. Exceeding this value may cause the retrieval to timeout or fail depending on the number of SPECTRA fields in `include`. 
</div>

In [ ]:
client.retrieve_by_specid?

#### Use the specIDs to retrieve records from SPARCL:
At the Data Lab, the `sparcl.main` table contains the `specid` column. This survey-specific identifier corresponds to `TARGETID` for DESI and to `specObjID` for SDSS/BOSS. As a consequence, there may be more spectra than number of unique TARGETID's given that the DESI healpix-coadd spectra are only uniquely defined by `TARGETID, SURVEY, PROGRAM`. We show an example case below to remove duplicates and only keep the best (specprimary) spectrum per object.

We first need to convert the Pandas Series of specIDs into an int list to work with SPARCL. Then we can pass the int list to the `client.retrieve_by_specid()` method.

In [ ]:
# Define the fields to include in the retrieve function
inc = ['sparcl_id', 'specid', 'data_release', 'redshift', 'flux',
       'wavelength', 'model', 'ivar', 'mask', 'ra', 'dec', 'specprimary']

In [ ]:
specids = [int(s) for s in found_df_II['targetid']]

In [ ]:
%%time
results_II = client.retrieve_by_specid(specid_list=specids, include=inc, dataset_list=['DESI-DR1'], limit=1500)
results_II.info

In [ ]:
## Convert the output into a Pandas DataFrame
df_II = pd.json_normalize(results_II.records)

In [ ]:
print(len(df_II))
print(len(df_II[df_II['specprimary']]))

In [ ]:
## Keep only primary spectra
df_II = df_II[df_II['specprimary']]
print(len(df_II))

<a class="anchor" id="reorder"></a>
## Reorder spectrum records
The ordering of the results of `client.retrieve()` is not necessarily sorted to match the order of the IDs in the argument passed to the method. We show how to join the records stored as Pandas Dataframe with the input table.


In [ ]:
ids_II = list(found_df_II['targetid'][:6])
print(f"Order of SPARCL IDs originally:\n{ids_II}\n")

res = client.retrieve_by_specid(specid_list=specids[:6])
res_ids = [f.specid for f in res.records]
print(f"Order of SPARCL IDs after passing to retrieve():\n{res_ids}\n")

Next, we will work with our DataFrames for our sample (`found_df_II`) and the returned spectra (`df_II`) to join them into a combined DataFrame. Let's check the columns to compare column names and the IDs.

In [ ]:
found_df_II.columns

In [ ]:
df_II.columns

In [ ]:
## Rename "specid" to "targetid" which is appropriate when all spectra from DESI
# (REMINDER: specid=targetid for DESI; specid=specobjid for SDSS/BOSS)
df_II = df_II.rename(columns={"specid": "targetid"})

In [ ]:
## Join the DataFrames into one
df_join = found_df_II.join(df_II.set_index('targetid'), on='targetid', how='left')

df_join[:2]

<a class="anchor" id="convert"></a>
## Convert retrieved output to Spectrum1D object

#### Spectrum1D:
When applicable, the units for each field are documented on the [Fields tab of the astrosparcl website](https://astrosparcl.datalab.noirlab.edu/sparc/sfc/)

In [ ]:
%%time
# We create a list of Spectrum1D objects by iterating over the rows (r)
# from the reordered dataframe (df_join)
specs = [Spectrum1D(spectral_axis = r.wavelength*u.AA,
                    flux = np.array(r.flux)* 10**-17 * u.Unit('erg cm-2 s-1 AA-1'),
                    uncertainty = InverseVariance(np.array(r.ivar)),
                    redshift = r.redshift,
                    mask = r.mask)
        for r in df_join.itertuples()]
specs[0]

In [ ]:
# Plot a couple of examples
f, ax = plt.subplots()
for i in range(3):
    valid = specs[i].mask==0
    ax.step(specs[i].spectral_axis[valid], specs[i].flux[valid])
plt.title('Observed Frame')
plt.xlabel('Observed $\lambda\ [\AA]$')
plt.ylabel('$f_{\lambda}$ $(10^{-17}$ $erg$ $s^{-1}$ $cm^{-2}$ $\AA^{-1})$')
plt.show()

# Same two spectra in rest-frame
f, ax = plt.subplots()
for i in range(3):
    valid = specs[i].mask==0
    ax.step(specs[i].spectral_axis[valid]/(1.+specs[i].redshift), specs[i].flux[valid]*(1.+specs[i].redshift))
plt.title('Rest Frame')
plt.xlabel('Rest $\lambda\ [\AA]$')
plt.ylabel('$f_{\lambda}$ $(10^{-17}$ $erg$ $s^{-1}$ $cm^{-2}$ $\AA^{-1})$')
plt.show()

<a class="anchor" id="plot"></a>
## Plot spectra function

In [ ]:
def plot_spec(index, results):
    """
    Pass an index value and the output from using client.retrieve()
    to plot the spectrum at the specified index.
    """

    record = results.loc[index]

    # We renamed specid to targetid for DESI (specid=specobjid for SDSS/BOSS)
    specid = record.targetid
    data_release = record.data_release

    # Spectra quantities
    wavelength = record.wavelength
    model = record.model
    flux = record.flux
    # Special case with mask (.mask is a reserved syntax)
    mask = results['mask'].loc[index]

    # Round the redshift, RA, Dec to a reasonable number of significant digits
    redshift = np.round(record.redshift, 4)
    ra = np.round(record.ra, 7)
    dec = np.round(record.dec, 7)

    plt.title(f"Data Set = {data_release}\n"
              f"SpecID = {specid}\n"
              f"Redshift = {redshift}\n"
              f"RA = {ra}\n"
              f"Dec = {dec}", loc='left')
    plt.xlabel('$\lambda\ [\AA]$')
    plt.ylabel('$f_{\lambda}$ $(10^{-17}$ $erg$ $s^{-1}$ $cm^{-2}$ $\AA^{-1})$')

    # Plot unsmoothed spectrum in grey
    plt.plot(wavelength[mask==0], flux[mask==0], color='k', alpha=0.2, label='Unsmoothed spectrum')

    # Overplot spectrum smoothed using a 1-D Gaussian Kernel in black
    plt.plot(wavelength[mask==0], convolve(flux[mask==0], Gaussian1DKernel(5)), color='k', label='Smoothed spectrum')

    # Overplot the model spectrum in red
    plt.plot(wavelength[mask==0], model[mask==0], color='r', label='Model spectrum')

    plt.legend()
    plt.show()

In [ ]:
plot_spec(index=3, results=df_join)

<a class="anchor" id="additionalacknowledgments"></a>
## Additional acknowledgments

If you use **SPARCL jointly with the Astro Data Lab platform** (via JupyterLab, command-line, or web interface) in your published research, please include this text below in your paper's Acknowledgments section:

_This research uses services or data provided by the SPectra Analysis and Retrievable Catalog Lab (SPARCL) and the Astro Data Lab, which are both part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

If you are using the Astro Data Lab in your research, please cite the following papers:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Astro Data Lab overview: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411

If you are referring to the Data Lab JupyterLab / Jupyter Notebooks, cite:

* Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097

If publishing in a AAS journal, also add the keyword: `\facility{Astro Data Lab}`